In [2]:
import pandas as pd
from fastparquet import ParquetFile
import duckdb
import re

from datetime import datetime, timedelta


In [ ]:
stkr=  ParquetFile("../datasets/sticker.parquet").to_pandas()

In [136]:
target = 'type'

In [5]:
pvt = stkr.pivot(values='volume', index='item', columns = 'date').reset_index()

In [140]:
items = duckdb.query('select * from pvt join (select distinct type, item from stkr) as t2 on t2.item = pvt.item').df()

In [ ]:
cols = list(items.columns[-1000:-20])
cols

In [153]:
stats = items[cols + [target]].fillna(0).drop(target, axis=1).T.describe().T

In [154]:
stats[target] = items[target]
stats["item"] = items["item"]
stats

,count,mean,std,min,25%,50%,75%,max,type,item
0,980.0,0.034694,0.307915,0.0,0.0,0.0,0.00,6.0,Autograph,Sticker | Hobbit (Foil) | London 2018
1,980.0,0.210204,0.713719,0.0,0.0,0.0,0.00,9.0,Autograph,Sticker | HooXi (Gold) | Paris 2023
2,980.0,19.751020,52.402984,0.0,0.0,1.0,15.25,797.0,Autograph,Sticker | HooXi | Copenhagen 2024
3,980.0,0.316327,1.012657,0.0,0.0,0.0,0.00,15.0,Other,Sticker | Hypnotic Surf K (Foil)
4,980.0,0.010204,0.110241,0.0,0.0,0.0,0.00,2.0,Autograph,Sticker | Kaze (Foil) | Boston 2018
...,...,...,...,...,...,...,...,...,...,...
6588,980.0,6.171429,15.768732,0.0,0.0,0.0,4.00,205.0,Autograph,Sticker | tabseN | Shanghai 2024
6589,980.0,7.309184,17.008419,0.0,0.0,0.0,5.00,240.0,Autograph,Sticker | kyxsan (Holo) | Shanghai 2024
6590,980.0,21.969388,47.022489,0.0,0.0,1.0,12.25,272.0,Team,Sticker | HEROIC (Holo) | Shanghai 2024
6591,980.0,6.485714,15.797195,0.0,0.0,0.0,4.00,172.0,Autograph,Sticker | ICY (Glitter) | Shanghai 2024


In [155]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [156]:
train, test = train_test_split(stats, test_size=0.2, random_state=41)

In [157]:

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(train.drop([target, 'item'], axis=1), train[target])
classes = model.predict(test.drop([target, 'item'], axis=1))
multi_target_forest = MultiOutputClassifier(model, n_jobs=2)

In [158]:
out = pd.DataFrame(classes, columns=['predicted'])
out['actual'] = test[target].reset_index(drop=True)
out['item'] = test['item'].reset_index(drop=True)

In [159]:

for t in  out["actual"].unique():
    important = out[(out["actual"] == t) | (out["predicted"] == t)].drop('item', axis=1)
    TP = len(important[important == t].dropna())
    TN = len(out) - len(important)
    FN = len(important[(important['actual'] == t) & (important['predicted'] != t)])
    FP = len(important[(important['actual'] != t) & (important['predicted'] == t)])
    print(t, TP, FP, FN, TN)

Autograph 888 206 67 158
Other 68 19 61 1171
Team 60 77 167 1015
Event 0 1 8 1310


In [70]:
autographs = duckdb.query("select * from stkr where type = 'Autograph'").df()
autographs.head()

,date,price,volume,item,type,effect,market_hash_name,rarity,tournament_id,tournament,team_id,team_tag,team,image_url,rarity_color
0,2019-08-14 01:00:00,0.818,35,Sticker | Aleksib (Foil) | Berlin 2019,Autograph,Foil,Sticker | Aleksib (Foil) | Berlin 2019,Remarkable,16.0,2019 StarLadder Berlin,84.0,ENCE,ENCE,https://community.akamai.steamstatic.com/econo...,purple
1,2019-08-15 01:00:00,0.745,127,Sticker | Aleksib (Foil) | Berlin 2019,Autograph,Foil,Sticker | Aleksib (Foil) | Berlin 2019,Remarkable,16.0,2019 StarLadder Berlin,84.0,ENCE,ENCE,https://community.akamai.steamstatic.com/econo...,purple
2,2019-08-16 01:00:00,0.789,54,Sticker | Aleksib (Foil) | Berlin 2019,Autograph,Foil,Sticker | Aleksib (Foil) | Berlin 2019,Remarkable,16.0,2019 StarLadder Berlin,84.0,ENCE,ENCE,https://community.akamai.steamstatic.com/econo...,purple
3,2019-08-17 01:00:00,0.893,57,Sticker | Aleksib (Foil) | Berlin 2019,Autograph,Foil,Sticker | Aleksib (Foil) | Berlin 2019,Remarkable,16.0,2019 StarLadder Berlin,84.0,ENCE,ENCE,https://community.akamai.steamstatic.com/econo...,purple
4,2019-08-18 01:00:00,0.826,55,Sticker | Aleksib (Foil) | Berlin 2019,Autograph,Foil,Sticker | Aleksib (Foil) | Berlin 2019,Remarkable,16.0,2019 StarLadder Berlin,84.0,ENCE,ENCE,https://community.akamai.steamstatic.com/econo...,purple


In [71]:
names = duckdb.query("select distinct item from autographs where type = 'Autograph'").df()

In [77]:
parsed = {name[0]: re.match(r'Sticker \| (([^\(|\|])+)',name[0]).group(1).strip() for name in names.values.tolist()}

mappings = pd.Series(parsed).to_frame().reset_index()

mappings.columns = ["sticker_name", 'player']

duckdb.query("select * from autographs join mappings on autographs.item = mappings.sticker_name").df().drop(
    columns=['type', 'market_hash_name', 'image_url', 'sticker_name'], axis=1
).to_parquet('../autographs.parquet')

In [3]:
match =  ParquetFile("../matches.parquet").to_pandas()

In [7]:
pvt = duckdb.query('select price, volume, date, item from stkr where tournament is not null').df().pivot(index='item', columns = 'date', values = 'price')
pvt =  pvt.reset_index()

date,2014-08-05 01:00:00,2014-08-06 01:00:00,2014-08-07 01:00:00,2014-08-08 01:00:00,2014-08-09 01:00:00,2014-08-10 01:00:00,2014-08-11 01:00:00,2014-08-12 01:00:00,2014-08-13 01:00:00,2014-08-14 01:00:00,...,2025-11-12 00:00:00,2025-11-12 01:00:00,2025-11-12 02:00:00,2025-11-12 03:00:00,2025-11-12 04:00:00,2025-11-12 05:00:00,2025-11-12 06:00:00,2025-11-12 07:00:00,2025-11-12 08:00:00,2025-11-12 09:00:00
item,,,,,,,,,,,,,,,,,,,,,
Sticker | 3DMAX (Glitter) | Shanghai 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.538,0.555,0.555,0.54,NaN,0.549,0.54,0.540,0.544,NaN
Sticker | 3DMAX (Gold) | Shanghai 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.959,NaN,NaN,NaN,2.95,NaN,NaN,NaN,NaN,NaN
Sticker | 3DMAX (Holo) | Shanghai 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.780,NaN,NaN,NaN,NaN
Sticker | 3DMAX | Shanghai 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.030,0.040,0.030,0.03,0.03,0.030,0.03,0.040,0.030,NaN
Sticker | 910 (Glitter) | Shanghai 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sticker | zorte | Paris 2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sticker | ztr (Glitter) | Shanghai 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.050,NaN,NaN,NaN,NaN,NaN,0.035,NaN,NaN
Sticker | ztr (Gold) | Shanghai 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
pvt_price = duckdb.query('select * from stkr where team_tag is not null').df().pivot(index='item', columns = 'date', values = 'price').reset_index()

match =  ParquetFile("../matches.parquet").to_pandas()
match['date'] = pd.to_datetime(match['date'], format='mixed')

match

,date,event,event-id,winner,loser,winner-score,loser-score,stats
0,2015-03-15,ESL One Katowice 2015,1611,Fnatic,Ninjas in Pyjamas,2,1,https://www.hltv.org/matches/2294629/fnatic-vs...
1,2015-03-14,ESL One Katowice 2015,1611,Ninjas in Pyjamas,Team EnVyUs,2,0,https://www.hltv.org/matches/2294628/envy-vs-n...
2,2015-03-14,ESL One Katowice 2015,1611,Fnatic,Virtus.Pro,2,0,https://www.hltv.org/matches/2294627/fnatic-vs...
3,2015-03-14,ESL One Katowice 2015,1611,Ninjas in Pyjamas,TSM Kinguin,2,1,https://www.hltv.org/matches/2294626/ninjas-in...
4,2015-03-13,ESL One Katowice 2015,1611,Virtus.Pro,Keyd Stars,2,1,https://www.hltv.org/matches/2294625/virtuspro...
...,...,...,...,...,...,...,...,...
691,2024-03-21,PGL CS2 Major Copenhagen 2024,7148,Complexity Gaming,paiN Gaming,13,9,https://www.hltv.org/matches/2370633/complexit...
692,2024-03-21,PGL CS2 Major Copenhagen 2024,7148,Eternal Fire,Vitality,13,10,https://www.hltv.org/matches/2370630/vitality-...
693,2024-03-21,PGL CS2 Major Copenhagen 2024,7148,Imperial Esports,Virtus.Pro,13,6,https://www.hltv.org/matches/2370631/virtuspro...
694,2024-03-21,PGL CS2 Major Copenhagen 2024,7148,MOUZ,ECSTATIC,13,11,https://www.hltv.org/matches/2370628/mouz-vs-e...


In [14]:
pvt_price.reset_index(inplace=True)

In [ ]:
event_wins = match[['event', 'winner']]
top_3  = duckdb.query("""
    select event, winner from (SELECT
        event,
        winner,
        win_count,
        RANK() OVER (
            PARTITION BY event
            ORDER BY win_count DESC
        ) AS rank
    FROM (
        SELECT
            event,
            winner,
            COUNT(*) AS win_count
        FROM event_wins
        GROUP BY event, winner
    ))
        WHERE RANK <= 3
""").df()
top_3

In [111]:
lengths = duckdb.query("select event, min(date) as start_date, max(date) as end_date from match group by event").df()

In [112]:
data = {}
for event in match["event"].unique():
    timeframe = lengths.set_index('event').loc[event]
    teams = duckdb.query(f"select distinct winner from match where event = '{event}' UNION select distinct loser from match where event = '{event}'").df()

    team_stkrs = duckdb.query('select distinct item from stkr where team in (select winner from teams)').df()
    stkr_data = duckdb.query('select * from pvt_price where item in (select item from team_stkrs)').df()

    stkr_data = stkr_data[stkr_data.min().dropna().index]
    t = stkr_data.T.reset_index().iloc[1:]
    t['index'] = pd.to_datetime(t['index'])
    t = duckdb.query(f"""select * from t where index > '{str(timeframe['start_date'].date() - timedelta(50))}' and  index < '{str(timeframe['end_date'])}'""").df().T
    t.columns = list(t.loc['index'])
    t = t.iloc[1:]
    t['item'] = list(stkr_data['item'])
    t = t[t.drop('item',axis=1).T.isnull().apply(lambda x: not all(x))]
    stkr_data = duckdb.query('select * from t join (select distinct team, item from stkr) as t2 on t2.item = t.item').df().drop('item_1', axis=1)
    n = stkr_data.drop(['item', 'team'], axis=1)
    zs = (n.sub(n.mean(axis=1),axis=0)).div(n.std(axis=1), axis=0)
    zs['team'] = stkr_data['team']
    zm = zs.groupby('team').mean().reset_index()
    zm['top_team'] = zm['team'].isin(top_3[top_3['event'] == event]['winner'])
    data[event] = zm[list(zm.columns[-21:]) + ["team"]]

In [118]:
from sklearn.ensemble import RandomForestClassifier
predictions = []
for event in data.keys():
    if data[event].empty:
        print(event, "is empty")
        continue
    d = []
    for e in data.keys():
        if e is not event:
            d.append(pd.DataFrame(data[e].to_numpy()))
    d = pd.concat(d).dropna()
    if len(d.columns) != len(data[event].columns):
        print(event, "Has col mismatch")
        continue
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(d.drop([20,21], axis=1).astype(float), d[20].astype(bool))
    pred = model.predict(pd.DataFrame(data[event].drop(['team', 'top_team'], axis=1).to_numpy()).astype(float))
    predictions.append((list(pred), list(data[event]['top_team'])))

2014 EMS One Katowice is empty
2014 ESL One Cologne Has col mismatch


In [129]:
preds = pd.concat([pd.DataFrame(p) for p in predictions], axis=1).T
preds.columns = ["prediction", "actual"]

,prediction,actual
0,False,False
1,False,True
2,False,False
3,True,False
4,False,False
...,...,...
10,False,False
11,False,False
12,False,False
13,False,True


In [141]:
TP = len(preds[preds['prediction'] & preds['actual']])
TN = len(preds[(preds['prediction'] == False) & (preds['actual'] == False)])
FP = len(preds[(preds['prediction']) & (preds['actual'] == False)])
FN = len(preds[(preds['prediction'] == False) & (preds['actual'])])
print(TP, FP, FN, TN)

acc = (TP + TN) / (TP + TN + FP + FN)
prec = TP / (TP + FP)
rec = TP / (TP + FN)
f1 = 2 * prec * rec / (prec + rec)

print("Accuracy", acc)
print("Precision", prec)
print("Recall", rec)
print("F1", f1)

33 14 38 180
Accuracy 0.8037735849056604
Precision 0.7021276595744681
Recall 0.4647887323943662
F1 0.559322033898305
